In [2]:
df1 = spark.range(2, 10000000, 2)
df2 = spark.range(2, 10000000, 4)
step1 = df1.repartition(5)
step12 = df2.repartition(6)
step2 = step1.selectExpr("id * 5 as id")
step3 = step2.join(step12, ["id"])
step4 = step3.selectExpr("sum(id)")

step4.collect()

[Row(sum(id)=2500000000000)]

In [3]:
step4.explain()

== Physical Plan ==
*(7) HashAggregate(keys=[], functions=[sum(id#10L)])
+- Exchange SinglePartition
   +- *(6) HashAggregate(keys=[], functions=[partial_sum(id#10L)])
      +- *(6) Project [id#10L]
         +- *(6) SortMergeJoin [id#10L], [id#6L], Inner
            :- *(3) Sort [id#10L ASC NULLS FIRST], false, 0
            :  +- Exchange hashpartitioning(id#10L, 200)
            :     +- *(2) Project [(id#4L * 5) AS id#10L]
            :        +- Exchange RoundRobinPartitioning(5)
            :           +- *(1) Range (2, 10000000, step=2, splits=8)
            +- *(5) Sort [id#6L ASC NULLS FIRST], false, 0
               +- Exchange hashpartitioning(id#6L, 200)
                  +- Exchange RoundRobinPartitioning(6)
                     +- *(4) Range (2, 10000000, step=4, splits=8)
